# Β2.ii) Ομαδοποίηση με K-means Clustering

Αυτό το notebook εκτελεί την ομαδοποίηση κειμένων με K-means clustering όπως περιγράφεται στην ενότητα Β2.ii της εργασίας.

**Κύρια Χαρακτηριστικά:**
- **Μοντέλο**: K-means clustering
- **Feature Extraction**: TF-IDF vectorization  
- **Dataset**: "DominusTea/GreekLegalSum" από το Hugging Face Hub
- **Στόχος**: Ομαδοποίηση των νομικών κειμένων σε clusters

**Διαδικασία:**
1. Εύρεση του βέλτιστου αριθμού clusters (K) με χρήση evaluation metrics
2. Εφαρμογή του K-means με το επιλεγμένο K
3. Ανάλυση και αξιολόγηση των αποτελεσμάτων

In [ ]:
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples, normalized_mutual_info_score, adjusted_rand_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.simplefilter(action='ignore', category=FutureWarning)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully")

## 1. Παραμετροποίηση του Πειράματος

Ορισμός των βασικών παραμέτρων για την ομαδοποίηση με K-means clustering.

In [ ]:
# --- Configuration για K Selection ---
TEXT_COLUMN_TO_USE = 'summary'  # 'summary' or 'text'
# Χρησιμοποιούμε 'summary' γιατί είναι γενικά πιο καθαρό και λιγότερο υπολογιστικά εντατικό

VECTORIZATION_METHOD = 'tfidf' # TF-IDF είναι συνηθισμένη επιλογή για K-means
MIN_K_TO_TEST = 2           # Ελάχιστη τιμή K για test
MAX_K_TO_TEST = 50          # Μέγιστη τιμή K για test (μειωμένο για notebook)
K_STEP = 2                  # Βήμα για τις τιμές K
RANDOM_STATE = 42           # Για αναπαραγωγιμότητα

# --- Configuration για Final K-means ---
CHOSEN_K = 21               # Θα το ορίσουμε μετά την ανάλυση των metrics
OUTPUT_CSV_FILE = '../documents_with_clusters.csv'  # Αρχείο για αποθήκευση αποτελεσμάτων

print(f"Configuration:")
print(f"- Text column: {TEXT_COLUMN_TO_USE}")
print(f"- Vectorization method: {VECTORIZATION_METHOD}")
print(f"- K range: {MIN_K_TO_TEST} to {MAX_K_TO_TEST} (step: {K_STEP})")
print(f"- Random state: {RANDOM_STATE}")

## 2. Φόρτωση και Προετοιμασία Δεδομένων

Φορτώνουμε το dataset "DominusTea/GreekLegalSum" και το προετοιμάζουμε για την ανάλυση clustering.

In [ ]:
def load_and_prepare_data():
    """Loads the dataset and prepares it for analysis."""
    print("Loading the 'DominusTea/GreekLegalSum' dataset...")
    dataset_hf = None
    try:
        dataset_hf = load_dataset("DominusTea/GreekLegalSum", trust_remote_code=True)
        print("Dataset object loaded successfully from Hugging Face Hub.")
    except Exception as e:
        print(f"Error loading dataset from Hugging Face Hub: {e}")
        return None

    df = None
    if dataset_hf:
        dataset_split = dataset_hf.get('train')
        if dataset_split:
            print("Using 'train' split.")
            df = dataset_split.to_pandas()
        else:
            print("'train' split not found. Trying the first available split.")
            available_splits = list(dataset_hf.keys())
            if available_splits:
                split_to_use = available_splits[0]
                print(f"Attempting to use the first available split: '{split_to_use}'")
                try:
                    df = dataset_hf[split_to_use].to_pandas()
                except Exception as e:
                    print(f"Error converting split '{split_to_use}' to Pandas DataFrame: {e}")
            else:
                print("No splits available in the loaded dataset object.")
    
    if df is None:
        print("DataFrame could not be created.")
        return None

    print(f"\nDataFrame created. Shape: {df.shape}")
    
    # Αντικατάσταση NaN values με κενό string
    df[TEXT_COLUMN_TO_USE] = df[TEXT_COLUMN_TO_USE].fillna('')
    
    print(f"Data prepared. Using column '{TEXT_COLUMN_TO_USE}' for clustering.")
    return df

# Φόρτωση δεδομένων
df = load_and_prepare_data()
if df is not None:
    print(f"\nDataset info:")
    print(f"- Shape: {df.shape}")
    print(f"- Columns: {list(df.columns)}")
    print(f"- Non-null values in '{TEXT_COLUMN_TO_USE}': {df[TEXT_COLUMN_TO_USE].notna().sum()}")
else:
    print("Failed to load data. Cannot proceed.")

## 3. Διανυσματοποίηση Κειμένων

Μετατρέπουμε τα κείμενα σε αριθμητικές αναπαραστάσεις χρησιμοποιώντας TF-IDF vectorization.

In [ ]:
def vectorize_texts(texts_series, method='tfidf'):
    """Vectorizes texts using the specified method."""
    print(f"\nVectorizing texts using {method} on '{TEXT_COLUMN_TO_USE}' column...")
    if method == 'tfidf':
        vectorizer = TfidfVectorizer(
            max_df=0.90,    # Αγνοεί terms που εμφανίζονται σε >90% των documents
            min_df=5,       # Αγνοεί terms που εμφανίζονται σε <5 documents
            ngram_range=(1, 2),  # Unigrams και bigrams
            stop_words=None      # Δεν χρησιμοποιούμε predefined stop words για ελληνικά
        )
        X = vectorizer.fit_transform(texts_series)
        print(f"TF-IDF matrix shape: {X.shape}")
        print(f"Number of features: {len(vectorizer.get_feature_names_out())}")
        return X, vectorizer
    else:
        raise ValueError(f"Unsupported vectorization method: {method}")

# Διανυσματοποίηση
if df is not None:
    X, vectorizer = vectorize_texts(df[TEXT_COLUMN_TO_USE], method=VECTORIZATION_METHOD)
    print(f"\nVectorization completed.")
    print(f"Feature matrix shape: {X.shape}")
    print(f"Matrix sparsity: {(1 - X.nnz / (X.shape[0] * X.shape[1])) * 100:.2f}%")

## 4. Εύρεση Βέλτιστου Αριθμού Clusters (K)

Εκτελούμε K-means για διάφορες τιμές του K και υπολογίζουμε evaluation metrics για να βρούμε τη βέλτιστη τιμή.

**Metrics που χρησιμοποιούμε:**
- **WCSS (Within-Cluster Sum of Squares)**: Για την Elbow Method
- **Silhouette Score**: Μετρά πόσο καλά διαχωρισμένα είναι τα clusters
- **NMI (Normalized Mutual Information)**: Σύγκριση με τα ground truth labels (case_category, case_tags)
- **ARI (Adjusted Rand Index)**: Άλλη μετρική σύγκρισης με ground truth

In [ ]:
def run_kmeans_and_evaluate(X, df, min_k, max_k, k_step):
    """Runs K-means for a range of K and calculates evaluation metrics."""
    print(f"\nRunning K-means for K from {min_k} to {max_k} with a step of {k_step}...")
    
    k_values = list(range(min_k, max_k + 1, k_step))
    if not k_values: 
        if max_k >= min_k:
             k_values = [min_k] 
        else: 
            print(f"max_k ({max_k}) is less than min_k ({min_k}), K-means cannot be run.")
            return [], [], [], [], [], [], [], []

    wcss = []
    silhouette_scores_micro = []
    silhouette_scores_macro = []
    nmi_category_scores = []
    nmi_tags_scores = []
    ari_category_scores = [] 
    ari_tags_scores = []   

    true_labels_category_full = df['case_category']
    true_labels_tags_full = df['case_tags'] 

    for k in k_values:
        print(f"  Processing K={k}...")
        kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=RANDOM_STATE)
        cluster_labels = kmeans.fit_predict(X)

        # WCSS (Within-Cluster Sum of Squares)
        wcss.append(kmeans.inertia_)

        # Silhouette Score (micro average)
        s_micro = silhouette_score(X, cluster_labels, metric='euclidean')
        silhouette_scores_micro.append(s_micro)

        # Silhouette Score (macro average - per cluster average)
        sample_s_values = silhouette_samples(X, cluster_labels, metric='euclidean')
        cluster_avg_s_values = []
        for i in range(k): 
            ith_cluster_s_values = sample_s_values[cluster_labels == i]
            if len(ith_cluster_s_values) > 0:
                cluster_avg_s_values.append(ith_cluster_s_values.mean())
        
        if cluster_avg_s_values:
            s_macro = np.mean(cluster_avg_s_values)
        else:
            s_macro = np.nan 
        silhouette_scores_macro.append(s_macro)
        
        # NMI και ARI για case_category
        valid_indices_category = true_labels_category_full.notna()
        if valid_indices_category.sum() >= 2: 
            nmi_cat = normalized_mutual_info_score(
                true_labels_category_full[valid_indices_category],
                cluster_labels[valid_indices_category]
            )
            nmi_category_scores.append(nmi_cat)
            ari_cat = adjusted_rand_score( 
                true_labels_category_full[valid_indices_category],
                cluster_labels[valid_indices_category]
            )
            ari_category_scores.append(ari_cat)
        else:
            nmi_category_scores.append(np.nan)
            ari_category_scores.append(np.nan)

        # NMI και ARI για case_tags
        valid_indices_tags = true_labels_tags_full.notna()
        if valid_indices_tags.sum() >= 2:
            nmi_tag = normalized_mutual_info_score(
                true_labels_tags_full[valid_indices_tags],
                cluster_labels[valid_indices_tags]
            )
            nmi_tags_scores.append(nmi_tag)
            ari_tag = adjusted_rand_score( 
                true_labels_tags_full[valid_indices_tags],
                cluster_labels[valid_indices_tags]
            )
            ari_tags_scores.append(ari_tag)
        else:
            nmi_tags_scores.append(np.nan)
            ari_tags_scores.append(np.nan)
            
    return k_values, wcss, silhouette_scores_micro, silhouette_scores_macro, nmi_category_scores, nmi_tags_scores, ari_category_scores, ari_tags_scores

# Εκτέλεση evaluation για διάφορες τιμές K
if df is not None and 'X' in locals():
    k_values, wcss, s_micro, s_macro, nmi_cat, nmi_tag, ari_cat, ari_tag = run_kmeans_and_evaluate(
        X, df, MIN_K_TO_TEST, MAX_K_TO_TEST, K_STEP
    )
    print(f"\nEvaluation completed for {len(k_values)} K values.")

## 5. Οπτικοποίηση Evaluation Metrics

Δημιουργούμε γραφήματα για τις διάφορες μετρικές αξιολόγησης που θα μας βοηθήσουν να επιλέξουμε το βέλτιστο K.

In [ ]:
def plot_evaluation_metrics(k_values, wcss, s_micro, s_macro, nmi_cat, nmi_tag, ari_cat, ari_tag):
    """Plots the evaluation metrics for choosing K."""
    if not k_values:
        print("No K values to plot. Skipping plotting.")
        return
        
    print("\nPlotting evaluation metrics...")
    plt.style.use('seaborn-v0_8-whitegrid') 
    fig, axs = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('K-means Clustering Evaluation Metrics for Optimal K Selection', fontsize=16)

    # Elbow Method (WCSS)
    axs[0, 0].plot(k_values, wcss, marker='o', linestyle='-', color='dodgerblue')
    axs[0, 0].set_title('Elbow Method (WCSS vs. K)', fontsize=14)
    axs[0, 0].set_xlabel('Number of Clusters (K)', fontsize=12)
    axs[0, 0].set_ylabel('WCSS (Inertia)', fontsize=12)
    axs[0, 0].grid(True, linestyle='--', alpha=0.7)

    # Silhouette Scores
    axs[0, 1].plot(k_values, s_micro, marker='s', linestyle='-', color='mediumseagreen', label='Micro Avg. Silhouette')
    axs[0, 1].plot(k_values, s_macro, marker='^', linestyle='--', color='darkorange', label='Macro Avg. Silhouette (per cluster avg.)')
    axs[0, 1].set_title('Silhouette Scores vs. K', fontsize=14)
    axs[0, 1].set_xlabel('Number of Clusters (K)', fontsize=12)
    axs[0, 1].set_ylabel('Silhouette Score', fontsize=12)
    axs[0, 1].legend(fontsize=10)
    axs[0, 1].grid(True, linestyle='--', alpha=0.7)

    # NMI Scores (Combined on one plot)
    axs[1, 0].plot(k_values, nmi_cat, marker='D', linestyle='-', color='crimson', label='NMI vs. Case Category')
    axs[1, 0].plot(k_values, nmi_tag, marker='p', linestyle='--', color='purple', label='NMI vs. Case Tags')
    axs[1, 0].set_title('NMI Scores vs. K', fontsize=14)
    axs[1, 0].set_xlabel('Number of Clusters (K)', fontsize=12)
    axs[1, 0].set_ylabel('NMI Score', fontsize=12)
    axs[1, 0].legend(fontsize=10)
    axs[1, 0].grid(True, linestyle='--', alpha=0.7)

    # ARI Scores (Combined on one plot)
    axs[1, 1].plot(k_values, ari_cat, marker='o', linestyle='-', color='teal', label='ARI vs. Case Category')
    axs[1, 1].plot(k_values, ari_tag, marker='x', linestyle='--', color='sienna', label='ARI vs. Case Tags')
    axs[1, 1].set_title('ARI Scores vs. K', fontsize=14)
    axs[1, 1].set_xlabel('Number of Clusters (K)', fontsize=12)
    axs[1, 1].set_ylabel('ARI Score', fontsize=12)
    axs[1, 1].legend(fontsize=10)
    axs[1, 1].grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout(rect=[0, 0, 1, 0.96]) 
    plt.show()

# Οπτικοποίηση των metrics
if 'k_values' in locals() and k_values:
    plot_evaluation_metrics(k_values, wcss, s_micro, s_macro, nmi_cat, nmi_tag, ari_cat, ari_tag)
    
    # Εκτύπωση των αποτελεσμάτων για ανάλυση
    print(f"\nDetailed Results:")
    print(f"{'K':<3} {'WCSS':<10} {'Silh_Micro':<12} {'Silh_Macro':<12} {'NMI_Cat':<10} {'NMI_Tag':<10} {'ARI_Cat':<10} {'ARI_Tag':<10}")
    print("-" * 85)
    for i, k in enumerate(k_values):
        print(f"{k:<3} {wcss[i]:<10.2f} {s_micro[i]:<12.4f} {s_macro[i]:<12.4f} "
              f"{nmi_cat[i]:<10.4f} {nmi_tag[i]:<10.4f} {ari_cat[i]:<10.4f} {ari_tag[i]:<10.4f}")
else:
    print("No evaluation results to plot.")

## 6. Επιλογή Βέλτιστου K και Τελική Ομαδοποίηση

Με βάση τα γραφήματα και τις μετρικές, επιλέγουμε το βέλτιστο K και εκτελούμε την τελική ομαδοποίηση.

**Κριτήρια επιλογής K:**
- **Elbow Method**: Αναζητούμε το σημείο όπου η μείωση του WCSS επιβραδύνεται
- **Silhouette Score**: Επιλέγουμε K που μεγιστοποιεί το silhouette score
- **NMI/ARI**: Εξετάζουμε την ταύτιση με τα ground truth labels
- **Ερμηνευσιμότητα**: Το K πρέπει να είναι λογικό για την εφαρμογή

In [ ]:
def run_final_kmeans(X, df_input, chosen_k_value):
    """Runs K-means for the chosen K, evaluates, and adds labels to DataFrame."""
    df = df_input.copy()  # Δουλεύουμε σε αντίγραφο για να μην τροποποιήσουμε το original
    print(f"\nRunning K-means for the chosen K = {chosen_k_value}...")
    
    kmeans_model = KMeans(n_clusters=chosen_k_value, init='k-means++', n_init='auto', random_state=RANDOM_STATE)
    cluster_labels = kmeans_model.fit_predict(X)

    # Προσθήκη cluster labels στο DataFrame
    df['cluster_id'] = cluster_labels
    print(f"\nCluster labels added to DataFrame in column 'cluster_id'.")

    # Ανάλυση των cluster sizes
    print("\n--- Cluster Sizes ---")
    cluster_sizes = df['cluster_id'].value_counts().sort_index()
    for cluster_id_val, size in cluster_sizes.items():
        print(f"  Cluster {cluster_id_val}: {size} documents ({size/len(df)*100:.1f}%)")
        
    # Υπολογισμός τελικών metrics
    final_silhouette = silhouette_score(X, cluster_labels, metric='euclidean')
    print(f"\nFinal Silhouette Score: {final_silhouette:.4f}")
    
    # Ανάλυση σχέσης με ground truth (αν υπάρχουν)
    valid_category_mask = df['case_category'].notna()
    if valid_category_mask.sum() > 1:
        final_nmi_cat = normalized_mutual_info_score(
            df.loc[valid_category_mask, 'case_category'],
            df.loc[valid_category_mask, 'cluster_id']
        )
        final_ari_cat = adjusted_rand_score(
            df.loc[valid_category_mask, 'case_category'],
            df.loc[valid_category_mask, 'cluster_id']
        )
        print(f"NMI with case_category: {final_nmi_cat:.4f}")
        print(f"ARI with case_category: {final_ari_cat:.4f}")
    
    return df, kmeans_model

# Εκτέλεση τελικής ομαδοποίησης
if df is not None and 'X' in locals():
    # Αν θέλουμε να αλλάξουμε το K με βάση τα αποτελέσματα, μπορούμε να το κάνουμε εδώ
    print(f"\nUsing K = {CHOSEN_K} for final clustering...")
    
    df_with_clusters, kmeans_model = run_final_kmeans(X, df, CHOSEN_K)
    
    print(f"\nFinal clustering completed with {CHOSEN_K} clusters.")

## 7. Ανάλυση και Χαρακτηρισμός των Clusters

Αναλύουμε τα περιεχόμενα των clusters και εξάγουμε χαρακτηριστικά για κάθε ομάδα.

In [ ]:
def analyze_clusters(df_clustered, vectorizer, kmeans_model, n_top_terms=10):
    """Analyzes the content of each cluster."""
    print(f"\n--- Cluster Content Analysis ---")
    
    # Εξαγωγή των top terms για κάθε cluster από τα centroids
    feature_names = vectorizer.get_feature_names_out()
    centroids = kmeans_model.cluster_centers_
    
    cluster_top_terms = {}
    
    for cluster_id in range(len(centroids)):
        # Παίρνουμε τα indices των top terms για αυτό το cluster
        top_indices = centroids[cluster_id].argsort()[-n_top_terms:][::-1]
        top_terms = [feature_names[i] for i in top_indices]
        top_weights = [centroids[cluster_id][i] for i in top_indices]
        
        cluster_top_terms[cluster_id] = list(zip(top_terms, top_weights))
        
        print(f"\nCluster {cluster_id} (Size: {(df_clustered['cluster_id'] == cluster_id).sum()}):")
        print("Top terms:", ", ".join([f"{term}({weight:.3f})" for term, weight in cluster_top_terms[cluster_id]]))
        
        # Παραδείγματα κειμένων από το cluster
        cluster_docs = df_clustered[df_clustered['cluster_id'] == cluster_id]
        if len(cluster_docs) > 0:
            print("Sample documents:")
            for i, (idx, row) in enumerate(cluster_docs.head(2).iterrows()):
                summary_text = row[TEXT_COLUMN_TO_USE][:200] + "..." if len(row[TEXT_COLUMN_TO_USE]) > 200 else row[TEXT_COLUMN_TO_USE]
                print(f"  {i+1}. {summary_text}")
    
    return cluster_top_terms

# Ανάλυση clusters
if 'df_with_clusters' in locals() and 'vectorizer' in locals() and 'kmeans_model' in locals():
    cluster_terms = analyze_clusters(df_with_clusters, vectorizer, kmeans_model, n_top_terms=10)

## 8. Οπτικοποίηση Cluster Distribution

Δημιουργούμε οπτικοποιήσεις για την κατανομή των clusters και τη σχέση τους με τα ground truth labels.

In [ ]:
def visualize_cluster_results(df_clustered):
    """Creates visualizations for cluster analysis."""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('K-means Clustering Results Analysis', fontsize=16)
    
    # 1. Cluster size distribution
    cluster_counts = df_clustered['cluster_id'].value_counts().sort_index()
    axes[0, 0].bar(cluster_counts.index, cluster_counts.values, color='lightblue', edgecolor='navy')
    axes[0, 0].set_title('Cluster Size Distribution')
    axes[0, 0].set_xlabel('Cluster ID')
    axes[0, 0].set_ylabel('Number of Documents')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Cluster sizes as pie chart
    axes[0, 1].pie(cluster_counts.values, labels=cluster_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 1].set_title('Cluster Size Proportions')
    
    # 3. Relationship with case_category (if available)
    valid_category_data = df_clustered[df_clustered['case_category'].notna()]
    if len(valid_category_data) > 0:
        # Heatmap των top categories ανά cluster
        category_cluster_counts = valid_category_data.groupby(['cluster_id', 'case_category']).size().unstack(fill_value=0)
        
        # Παίρνουμε μόνο τις top 10 categories για καλύτερη οπτικοποίηση
        top_categories = valid_category_data['case_category'].value_counts().head(10).index
        category_cluster_subset = category_cluster_counts[top_categories]
        
        sns.heatmap(category_cluster_subset.T, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0])
        axes[1, 0].set_title('Cluster vs Top Case Categories')
        axes[1, 0].set_xlabel('Cluster ID')
        axes[1, 0].set_ylabel('Case Category')
    else:
        axes[1, 0].text(0.5, 0.5, 'No case_category data available', 
                       horizontalalignment='center', verticalalignment='center')
        axes[1, 0].set_title('Cluster vs Case Categories (No Data)')
    
    # 4. Document length distribution per cluster
    if TEXT_COLUMN_TO_USE in df_clustered.columns:
        df_clustered['text_length'] = df_clustered[TEXT_COLUMN_TO_USE].str.len()
        
        # Box plot για text lengths ανά cluster
        cluster_lengths = []
        cluster_labels = []
        for cluster_id in sorted(df_clustered['cluster_id'].unique()):
            cluster_docs = df_clustered[df_clustered['cluster_id'] == cluster_id]
            cluster_lengths.append(cluster_docs['text_length'].values)
            cluster_labels.append(f'C{cluster_id}')
        
        axes[1, 1].boxplot(cluster_lengths, labels=cluster_labels)
        axes[1, 1].set_title('Text Length Distribution by Cluster')
        axes[1, 1].set_xlabel('Cluster ID')
        axes[1, 1].set_ylabel('Text Length (characters)')
        axes[1, 1].tick_params(axis='x', rotation=45)
    else:
        axes[1, 1].text(0.5, 0.5, 'Text length analysis not available', 
                       horizontalalignment='center', verticalalignment='center')
        axes[1, 1].set_title('Text Length Distribution (No Data)')
    
    plt.tight_layout()
    plt.show()

# Οπτικοποίηση αποτελεσμάτων
if 'df_with_clusters' in locals():
    visualize_cluster_results(df_with_clusters)

## 9. Αποθήκευση Αποτελεσμάτων

Αποθηκεύουμε το DataFrame με τα cluster assignments σε CSV αρχείο για περαιτέρω ανάλυση.

In [ ]:
def save_results(df_clustered, output_file):
    """Saves the DataFrame with cluster assignments to CSV."""
    try:
        # Δημιουργία του directory αν δεν υπάρχει
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Created directory: {output_dir}")

        # Αποθήκευση σε CSV
        df_clustered.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\nDataFrame with cluster assignments saved to '{os.path.abspath(output_file)}'")
        
        # Σύνοψη των αποτελεσμάτων
        print(f"\nResults Summary:")
        print(f"- Total documents: {len(df_clustered)}")
        print(f"- Number of clusters: {df_clustered['cluster_id'].nunique()}")
        print(f"- Output file: {os.path.abspath(output_file)}")
        
        return True
    except Exception as e:
        print(f"\nError saving DataFrame to CSV: {e}")
        return False

# Αποθήκευση αποτελεσμάτων
if 'df_with_clusters' in locals():
    success = save_results(df_with_clusters, OUTPUT_CSV_FILE)
    if success:
        print(f"\nClustering process completed successfully!")
        print(f"The file '{OUTPUT_CSV_FILE}' contains all documents with their assigned cluster IDs.")

## 10. Συμπεράσματα και Παρατηρήσεις

### Αποτελέσματα της Ομαδοποίησης

Βασισμένα στην ανάλυση που πραγματοποιήσαμε:

**Τεχνικά Χαρακτηριστικά:**
- **Dataset**: "DominusTea/GreekLegalSum" με χρήση του πεδίου 'summary'
- **Vectorization**: TF-IDF με παραμέτρους max_df=0.90, min_df=5, ngram_range=(1,2)
- **Clustering Algorithm**: K-means με k-means++ initialization
- **Επιλεγμένο K**: 21 clusters

**Αξιολόγηση των Αποτελεσμάτων:**
- **Silhouette Score**: Μετρά την ποιότητα των clusters σε όρους εσωτερικής συνοχής και διαχωρισμού
- **NMI/ARI vs Ground Truth**: Εξετάζει πόσο καλά τα clusters ταιριάζουν με τις υπάρχουσες κατηγορίες

**Παρατηρήσεις:**
1. **Cluster Distribution**: Η κατανομή των μεγεθών των clusters δείχνει...
2. **Θεματική Συνοχή**: Τα top terms κάθε cluster υποδηλώνουν...
3. **Σχέση με Ground Truth**: Η σύγκριση με τις υπάρχουσες κατηγορίες αποκαλύπτει...

**Πιθανές Βελτιώσεις:**
- Χρήση διαφορετικών vectorization τεχνικών (Word2Vec, Doc2Vec, BERT embeddings)
- Εφαρμογή άλλων clustering αλγορίθμων (Hierarchical, DBSCAN)
- Pre-processing βελτιώσεις (stemming, lemmatization για ελληνικά)
- Feature selection για μείωση της διαστατικότητας

Η ομαδοποίηση των νομικών κειμένων παρέχει πολύτιμες γνώσεις για την οργάνωση και κατηγοριοποίηση του νομικού περιεχομένου, και μπορεί να χρησιμοποιηθεί για περαιτέρω ανάλυση και εφαρμογές στον τομέα της νομικής επιστήμης.